# Pygame

---

Pygame je python knihovna (tedy ji musíme nainstalovat pomocí `pip install pygame` a naimportovat do našeho python souboru pomocí `import pygame`, tedy podoběn třeba jako `import random` pro náhodné generování nejen čísel). Pygame slouží pro vykreslování "okénkové" aplikace, nebudeme tedy už pracovat jen v konzoli a psát tam text, nicméně podíváme se na to, jak tvořit grafické rozhraní pro uživatele (GUI) a naučíme se, jak v tomto grafickém prostředí něco dělat, jelikož nám půjde o tvoření her, tak například ovládání hráče v prostředí. 

Začneme s tím, jak vůbec zobrazit okno:

In [ ]:
import pygame

# Inicializace Pygame
pygame.init()

# Nastavení velikosti okna (šířka, výška)
width, height = 800, 600
screen = pygame.display.set_mode((width, height))

# Titulek okna
pygame.display.set_caption("Pygame gameska")

# Herní smyčka - běží pořád dokola
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Vyplnění pozadí barvou (např. bílou)
    screen.fill((255, 255, 255))

    # Aktualizace obrazovky
    pygame.display.flip()

# Ukončení Pygame
pygame.quit()

Tento kód zobrazí bíle okno s popiskem "Pygame gameska" o velikosti 800x600 pixelů. Projděte si celý kód. Nejdřív importujeme Pygame, následovně inicializujeme (spouštíme) Pygame, následně nastavujme velikost okna, nastavujeme titulek okna a následuje herní smyčka, v této smyčce se vykresluje obrazovka (my ji vykreslujeme jen bílou), mohou se zde vypočítávát například pozice hráče a zároveň zde kontrolujeme zda uživatel neukončil hru, abychom pak mohli ukončit celou aplikaci.

Jak ale něco vykreslit? Základní geometrické útvary můžeme vykreslit pomocí `pygame.draw....`, které umístíme před příkaz `pygame.display.flip()`, který updatuje obrazovku. V Pygame funguje systém souřadnic (odpovídají pixelům obrazovky), tedy pokud chceme něco někam vykreslit, musíme uvést co a kam. Tedy např. `pygame.draw.rect(screen, (0, 0, 255), (100, 100, 200, 100))` vykreslí obdelník na obrazovku `screen` (máme ji tam jako proměnnou), s barvou v RGB `(0, 0, 255)` což je modrá, na pozici `100, 100` (střed útvaru) a s délkou a šířkou `200, 100`. Obdobně můžeme vykreslit kruh pomocí `pygame.draw.circle(screen, (255, 0, 0), (400, 300), 50)`, kde zase vykreslujeme na obrazovku `screen`, barva je červená `(255, 0, 0)`, se tředem na souřadnici `(400, 300)` a s poloměrem `50`. Nebo třeba `pygame.draw.polygon(screen, (0, 255, 0), [(200, 200), (250, 100), (300, 200)])` vykreslí zelený trojúhelník se vrcholy na souřadnících `(200, 200)`, `(250, 100)` a `(300, 200)`, pomocí `pygame.draw.polygon(...)` můžeme vykreslit i jiný mnohoúhelník, přidáme jen další souřadnici vrcholu a Pygame nám tvar spojí.

Už umíme vykreslovat tvary, ale chtělo by to i umět ovládat jejich pozici (pohybovat s nimi) pomocí WASD kláves. To můžeme udělat následovně:

In [ ]:
import pygame

# Inicializace Pygame
pygame.init()

# Nastavení velikosti okna (šířka, výška)
width, height = 800, 600
screen = pygame.display.set_mode((width, height))

# Titulek okna
pygame.display.set_caption("Pygame gameska")

# Pozice hráče - na začátku uprostřed
player_pos = pygame.Vector2(screen.get_width() / 2, screen.get_height() / 2)

# Herní smyčka - běží pořád dokola
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Vyplnění pozadí barvou (např. bílou)
    screen.fill((255, 255, 255))
    
    # Zjistíme co uživatel zmáčknul
    keys = pygame.key.get_pressed()
    if keys[pygame.K_w]: # Zmáčknul W?
        player_pos.y -= 1
    if keys[pygame.K_s]: # Zmáčknul S?
        player_pos.y += 1
    if keys[pygame.K_a]: # Zmáčknul A?
        player_pos.x -= 1
    if keys[pygame.K_d]: # Zmáčknul D?
        player_pos.x += 1

    # Vykreslení kruhu na pozici prayer_pos
    pygame.draw.circle(screen, (255, 0, 0), player_pos, 40)

    # Aktualizace obrazovky
    pygame.display.flip()

# Ukončení Pygame
pygame.quit()


Tento kód je stejný, jako pro vykreslení prázdené obrazovky, jen jsme přidali informaci o pozici uživatele jako dvojici hodnot (`pygame.Vector2(...)`), tuhle pozici měníme na z herní smyčky, kde si zíkáme pomocí `keys = pygame.key.get_pressed()` co uživatel zmáčknul za klávesy a následně pomocí `if keys[pygame.K_s]:` atd. kontrolujeme, zda zmáčknul klávesu pro pohyb a pokud ano, tak přičítáme nebo odčítáme k souřadnicím daný posun.